#COBRAme Tutorial

In [1]:
import cobrame
from cobrame.util import dogma
import cobra
import cobra.test
from collections import defaultdict

import warnings
warnings.filterwarnings('ignore')

## Adding metabolic reactions

COBRAme is constructed entirely over COBRApy. This means that ME reactions will have all of the same properties, methods, and functions as a COBRApy reaction. However, one key difference is that, in order to facilliate the template nature of many E-gene reactions, reactions are constructed and their components are manipulated via the use of the ProcessData classes. These act as information vessels for holding the information assocatied with a cellular process in simple, standard datatypes such as dictionaries and strings. 

### Start with generic reaction a <=> b
The below code creates metabolites 'a' and 'b' and adds them to the ME model. It then creates a StoichiometricData object which stores all of the stoichiometric information associated with the MetabolicReaction. This includes:
 - Reaction Stoichiometry
 - Lower and upper bound
 
Only one StoichiometricData object should be created for both reversible and irreversible reactions

In [2]:
# create empty ME-model
me = cobrame.MEmodel('test')
ijo = cobra.test.create_test_model('ecoli')

for met in ijo.metabolites:
    me.add_metabolites(met)

In [3]:
# "Translational capacity" of organism
me.global_info['kt'] =  4.5 #(in h-1)scott 2010, RNA-to-protein curve fit
me.global_info['r0'] =  0.087 #scott 2010, RNA-to-protein curve fit
me.global_info['k_deg'] =  0. #1.0/5. * 60.0  # 1/5 1/min 60 min/h # h-1

# Molecular mass of RNA component of ribosome
me.global_info['m_rr'] = 1700. # in kDa

# Average molecular mass of an amino acid
me.global_info['m_aa'] = 109. / 1000. #109. / 1000. # in kDa

# Proportion of RNA that is rRNA
me.global_info['f_rRNA'] = .86
me.global_info['m_nt'] = 324. / 1000. # in kDa
me.global_info['f_mRNA'] = .02

# tRNA associated global information
me.global_info['m_tRNA'] = 25000. / 1000. # in kDA
me.global_info['f_tRNA'] = .12

In [4]:
# create and add metabolites a and b to model as with COBRApy
a = cobrame.Component('a')
b = cobrame.Component('b')
me.add_metabolites([a,b])

# unique to COBRAme, construct a stoichiometric data object with the reaction information
data = cobrame.StoichiometricData('a_to_b', me)
stoichiometry = {'a':-1, 'b': 1}
data._stoichiometry = stoichiometry
data.lower_bound = -1000
data.upper_bound = 1000

The StoichiometricData for this reversible reaction is then assigned to two different MetabolicReactions (Due to the enzyme dilution constraint, all enzyme catalyzed reactions must be reverisble; more on this later). The MetabolicReaction require:
 - The associated StoichiometricData
 - True for reverse reactions, False for farward reactions
 - Enzyme $K_{eff}$ for reaction (discussed later, dafault=65)

These fields are then processed and the actual model reaction is created using the MetabolicReaction's update() function

In [5]:
# Create a forward ME Metabolic Reaction and associate the stoichiometric data to it
rxn = cobrame.MetabolicReaction('a_to_b_FWD_complex_ab')
me.add_reaction(rxn)
rxn.stoichiometric_data = data
rxn.reverse = False
# Update
rxn.update()

# Create a reverse ME Metabolic Reaction and associate the stoichiometric data to it
rxn = cobrame.MetabolicReaction('a_to_b_REV_complex_ab')
me.add_reaction(rxn)
rxn.stoichiometric_data = data
rxn.reverse = True
# Update
rxn.update()


print me.reactions.a_to_b_FWD_complex_ab.reaction
print me.reactions.a_to_b_REV_complex_ab.reaction

a --> b
b --> a


## Associate a_to_b forward and reverse reaction with a catalyzing enzyme
### Create ComplexData for enzyme
For COBRAme models, the reaction gene-protein-reaction rule (GPR) is replaced with a metabolite representing the synthesis of the enzyme(s) catalyzing a reaction. This metabolite is formed explicitly in a ME model by seperate reaction to transcribe the gene(s) and translate the protein(s) the compose the complex. These reactions will be added later.

ComplexData objects contain:
 - Subunit stoichiometry
 - Modification dictionary (discussed later)
 - Translocation dictionary (discussed later)
 
as well as a create_complex_formation() function to create the sythesis reaction following the naming conventions. It contains an update() function which incorporates changes in the ComplexData

In [6]:
data = cobrame.ComplexData('complex_ab', me)
data.stoichiometry = {'protein_a': 1, 'protein_b': 1}
data.create_complex_formation()
print me.reactions.formation_complex_ab.reaction

Created <TranslatedGene protein_a at 0x7f425ed8aed0> in <ComplexFormation formation_complex_ab at 0x7f425ed8a050>
Created <TranslatedGene protein_b at 0x7f425ed8afd0> in <ComplexFormation formation_complex_ab at 0x7f425ed8a050>
protein_a + protein_b --> complex_ab


### Associate enzyme with MetabolicReaction

The ComplexData object created in the previous cell can be incorporated into the MetabolicReaction using code below. Note: the update() function is required to apply the change.

In [7]:
me.reactions.a_to_b_FWD_complex_ab.complex_data = data
print me.reactions.a_to_b_FWD_complex_ab.reaction
me.reactions.a_to_b_FWD_complex_ab.update()
print me.reactions.a_to_b_FWD_complex_ab.reaction

me.reactions.a_to_b_REV_complex_ab.complex_data = data
print me.reactions.a_to_b_REV_complex_ab.reaction
me.reactions.a_to_b_REV_complex_ab.update()
print me.reactions.a_to_b_REV_complex_ab.reaction

a --> b
a + 4.27350427350427e-6*mu complex_ab --> b
b --> a
b + 4.27350427350427e-6*mu complex_ab --> a


The coefficient for complex_ab is determined by the expression $$\frac{\mu}{k_{eff}}$$ which in its entirety represents the dilution of an enzyme following a cell doubling. The coupling constraint can be summarized as followed
$$ 
\begin{align}
&v_{dilution,j} = \mu \sum_{i} \left( \frac{1}{k_{eff,i}} v_{usage,i} \right), & \forall j \in Enzyme
\end{align}
$$

Where $v_{usage,i}$ is the flux through the metabolic reaction, $\mu$ is the growth rate of the cell, and $k_{eff}$ is a kinetic parameter which conveys the productivity of the enzyme complex. Physically, it can be thought of as the number of reactions the enzyme can catalyze per cell division. 


By default the $k_{eff}$ for a MetabolicReaction is set to 65 but this can be changed using the code below.

In [8]:
me.reactions.a_to_b_FWD_complex_ab.keff = 1000
me.reactions.a_to_b_FWD_complex_ab.update()

# The forward and reverse direction can have differing keffs
print me.reactions.a_to_b_FWD_complex_ab.reaction
print me.reactions.a_to_b_REV_complex_ab.reaction

a + 2.77777777777778e-7*mu complex_ab --> b
b + 4.27350427350427e-6*mu complex_ab --> a


## Adding transcription and translation reactions
Here we take advantage of an additional subclass of ProcessData, called a SubreactionData object. This class is used to lump together processeses that occur as a result of many individual reactions, including translation elongation, ribosome formation, tRNA charging, etc. Since each of these steps often involve an enzyme that requires its own dilution coupling, this process allows these processes to be lumped into one reaction while still enabling each subprocess to be modified.

Below, we add the SubreactionData (excluding enzymes) for the addition of each amino acid using information from the E. coli codon table

### Add in the TranslationData for protein_a and protein_b
In order to add a TranslationData object to a ME model the user must additionally specifify the mRNA id and protein id of the translation reaction that will be added. This information as well as a nucleotide sequence is the only information required to add a translation reaction.

In [9]:
data = cobrame.TranslationData('a', me, 'RNA_a', 'protein_a')
data.nucleotide_sequence = "ATG" + "TTT" * 12 + "TAT" * 12 + "ACG" * 12 + "GAT" * 12 + "AGT" * 12 + "TGA"

### Add in the TranslationReaction for protein_a and protein_b
By associating the TranslationReaction with its corresponding TranslationData object and running the update function, COBRAme will create a reaction reaction for the nucleotide sequence given based on the organisms codon table and prespecified translation machinery.

In [10]:
sequence = "ATG" + "TTT" * 12 + "TAT" * 12 + "ACG" * 12 + "GAT" * 12 + "AGT" * 12 + "TGA"
RNA_a = cobrame.TranscribedGene('RNA_a')
me.add_metabolites(RNA_a)
RNA_a.nucleotide_sequence = sequence

In [11]:
rxn = cobrame.TranslationReaction('translation_a')
rxn.translation_data = data
me.add_reaction(rxn)
rxn.update()
print rxn.reaction

0.000498399634202103*mu + 0.000195123456790123 RNA_a + 12 asp__L_c + met__L_c + 12 phe__L_c + 12 ser__L_c + 12 thr__L_c + 12 tyr__L_c --> protein_a + 7.500606 protein_biomass


This reaction also produces a small amount of the a $biomass$ metabolite (constraint). This term has a coefficient corresponding to the molecular weight (in $kDA$) of the protein being translated. This constraint will be implemented into a $v_{biomasss\_dilution}$ reaction with the form: 
$$\mu \leq v_{biomass\_dilution} \leq \mu $$

This effectively is just a conservation of mass constraint saying that biomass must be diluted at the growth rate $\mu$ 

The coefficient for RNA_a represents 
$$
\begin{align}
& v_{dilution,j} \geq \frac{3}{\kappa_{\tau} c_{mRNA}} \cdot (\mu + \kappa_{\tau} r_0) v_{translation,j}  , & \forall j \in mRNA \\
\end{align}
$$
where:$\dots$

In [12]:
data = cobrame.TranscriptionData('TU_a',me,RNA_products={'RNA_a'})
data.nucleotide_sequence = sequence

In [13]:
rxn = cobrame.TranscriptionReaction('transcription_TU_a')
rxn.transcription_data = data
me.add_reaction(rxn)
rxn.update()
print rxn.reaction

86 atp_c + 38 ctp_c + 12 gtp_c + 186 h2o_c + 50 utp_c --> RNA_a + 186 h_c + 186 ppi_c
